## Libraries

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, LoggingHandler
from torch.utils.data import DataLoader
from sklearn.cluster import KMeans
from collections import Counter
from tqdm import tqdm
import numpy as np
import logging
import random
import pickle
import faiss
import torch
import json
import re
import os

In [ ]:
with open("dataset_2020.json", "r") as f:
    data = json.load(f)

len(data)

457

In [ ]:
data[0]

{'metadata': {'document_id': 'F2020001',
  'domain': '',
  'year': 2020,
  'journal_number': '001',
  'hijri_date': 'Dimanche 9 Joumada El Oula  1441',
  'gregorian_date': '5 janvier 2020',
  'document_link': 'https://www.joradp.dz/FTP/JO-FRANCAIS/2020/F2020001.pdf'},
 'content': "décret présidentiel n° 19-369 du aouel joumada el oula 1441 correspondant au 28 décembre 2019 déchargeant le ministre des affaires étrangères de l’intérim des fonctions de premier ministre. le président de la république, vu la constitution, notamment son article 91 5° et 6° vu le décret présidentiel n° 19-363 du 22 rabie ethani 1441 correspondant au 19 décembre 2019 chargeant le ministre des affaires étrangères de l’intérim du premier ministre décrète article 1er. m. sabri boukadoum, ministre des affaires étrangères est déchargé de l’intérim des fonctions de premier ministre. art. 2. le présent décret sera publié au journal officiel de la république algérienne démocratique et populaire. fait à alger, le aouel

In [4]:
documents = [entry["content"] for entry in data if entry["content"]]

In [ ]:
def chunk_text(text, size=512, stride=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words) - size + 1, stride):
        chunk = " ".join(words[i:i+size])
        chunks.append(chunk)
    return chunks


In [6]:
all_chunks = []
for i, doc in enumerate(documents):
    chunks = chunk_text(doc)
    print(f"{i} done")
    all_chunks.extend(chunks)

0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123

## Get embedding model

In [ ]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

2025-05-17 08:17:27.968552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747469848.154871      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747469848.206604      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
chunk_embeddings = model.encode(all_chunks, convert_to_numpy=True, show_progress_bar=True)

Batches:   0%|          | 0/3336 [00:00<?, ?it/s]

In [ ]:
print("Shape:", chunk_embeddings.shape)
print("Contains NaNs:", np.isnan(chunk_embeddings).any())

Shape: (106745, 384)
Contains NaNs: False


## Get clusters

In [ ]:
num_clusters = len(data) * 3 * 4 
clustering = KMeans(n_clusters=num_clusters, verbose=1, n_init="auto", random_state=42)
clustering.fit(chunk_embeddings)

Initialization complete
Iteration 0, inertia 217310.03125.
Iteration 1, inertia 162430.890625.
Iteration 2, inertia 157628.671875.
Iteration 3, inertia 155934.75.
Iteration 4, inertia 155129.90625.
Iteration 5, inertia 154680.34375.
Iteration 6, inertia 154412.75.
Iteration 7, inertia 154243.65625.
Iteration 8, inertia 154135.5.
Iteration 9, inertia 154063.1875.
Iteration 10, inertia 154012.65625.
Iteration 11, inertia 153973.9375.
Iteration 12, inertia 153941.6875.
Iteration 13, inertia 153920.84375.
Iteration 14, inertia 153902.59375.
Iteration 15, inertia 153888.921875.
Iteration 16, inertia 153878.65625.
Iteration 17, inertia 153872.6875.
Iteration 18, inertia 153866.15625.
Iteration 19, inertia 153860.578125.
Iteration 20, inertia 153856.9375.
Iteration 21, inertia 153853.59375.
Iteration 22, inertia 153851.25.
Iteration 23, inertia 153849.3125.
Iteration 24, inertia 153847.96875.
Iteration 25, inertia 153846.25.
Iteration 26, inertia 153845.171875.
Iteration 27, inertia 153844.57

KMeans(n_clusters=5484, n_init='auto', random_state=42, verbose=1)

In [ ]:
counts = Counter(clustering.labels_)
print(f"Min cluster size: {min(counts.values())}")
print(f"Max cluster size: {max(counts.values())}")
print(f"Clusters with < 3 chunks: {sum(1 for c in counts.values() if c < 3)}")

Min cluster size: 1
Max cluster size: 127
Clusters with < 3 chunks: 123


In [ ]:
clustered_chunks = {}
for idx, label in tqdm(enumerate(clustering.labels_), total=len(clustering.labels_)):
    clustered_chunks.setdefault(label, []).append(all_chunks[idx])

100%|██████████| 106745/106745 [00:00<00:00, 455368.90it/s]


## Embedding

In [ ]:
train_examples = []

# positive pairs (from the same cluster)
for cluster_chunks in tqdm(clustered_chunks.values(), desc="Generating positive pairs"):
    if len(cluster_chunks) < 2:
        continue
    for i in range(len(cluster_chunks) - 1):
        train_examples.append(InputExample(texts=[cluster_chunks[i], cluster_chunks[i + 1]], label=1.0))

# negative pairs (random chunks from different clusters)
for _ in tqdm(range(len(train_examples)), desc="Generating negative pairs"):
    chunk1 = random.choice(all_chunks)
    chunk2 = random.choice(all_chunks)
    train_examples.append(InputExample(texts=[chunk1, chunk2], label=0.0))


Generating negative pairs: 100%|██████████| 101261/101261 [00:00<00:00, 150329.80it/s]


## Load data

In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16, num_workers=0)

In [ ]:
print(type(train_examples))
print(type(train_examples[0]))
print(train_examples[0].label)

<class 'list'>
<class 'sentence_transformers.readers.InputExample.InputExample'>
1.0


In [ ]:
class ProgressLogger:
    def __init__(self, total_steps):
        self.total_steps = total_steps
        self.current_step = 0

    def on_step(self):
        self.current_step += 1
        if self.current_step % 1000 == 0 or self.current_step == self.total_steps:
            print(f"Training step {self.current_step}/{self.total_steps * 3}")

batches_per_epoch = len(train_dataloader)
total_steps = batches_per_epoch * 1 
progress_logger = ProgressLogger(total_steps)

# wrap the train loss to add a callback hook
class LossWithProgress(losses.CosineSimilarityLoss):
    def forward(self, sentence_features, labels):
        progress_logger.on_step()
        return super().forward(sentence_features, labels)

train_loss = LossWithProgress(model)

## Use GPU

In [ ]:
print(torch.cuda.is_available())

True


In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [22]:
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

True
0
Tesla P100-PCIE-16GB


In [24]:
print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())
print("Is CUDA available:", torch.cuda.is_available())

Torch version: 2.5.1+cu124
CUDA version: 12.4
cuDNN version: 90100
Is CUDA available: True


In [25]:
model.train()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [27]:
int(0.1 * len(train_dataloader) * 3)

3797

## fine-tuning the model

In [28]:
print("Starting training...")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=int(0.1 * len(train_dataloader) * 3),
    show_progress_bar=True,
    output_path=None
)

print("end training...")

Starting training...


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.072300
1000,0.052000
1500,0.047200
2000,0.045300
2500,0.045600
3000,0.045300
3500,0.043400
4000,0.045000
4500,0.042300
5000,0.040900


Training step 1000/37974
Training step 2000/37974
Training step 3000/37974
Training step 4000/37974
Training step 5000/37974
Training step 6000/37974
Training step 7000/37974
Training step 8000/37974
Training step 9000/37974
Training step 10000/37974
Training step 11000/37974
Training step 12000/37974
Training step 12658/37974
Training step 13000/37974
Training step 14000/37974
Training step 15000/37974
Training step 16000/37974
Training step 17000/37974
Training step 18000/37974
Training step 19000/37974
Training step 20000/37974
Training step 21000/37974
Training step 22000/37974
Training step 23000/37974
Training step 24000/37974
Training step 25000/37974
Training step 26000/37974
Training step 27000/37974
Training step 28000/37974
Training step 29000/37974
Training step 30000/37974
Training step 31000/37974
Training step 32000/37974
Training step 33000/37974
Training step 34000/37974
Training step 35000/37974
Training step 36000/37974
Training step 37000/37974
end training...


In [ ]:
model.save("fine-tuned-embedding-model-2020")